# Using EIGN's Laplacian and Convolution Operators

Here, we will explore in further detail how to use the Laplacian operators and the associated convolutions to build your own model with EIGN's layers.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Create a dummy graph

Let's create a dummy graph with some directed and undirected edges. We assign each edge with features that have an inherent direction (signed or orientation equivariant features) and some features that do not have an inherent direction (unsigned or orientation invariant features).

The signed features are represented with respect to some arbitrary edge orientation through their sign. The edge orientation is encoded through the `edge_index`: If it contains an edge `(u, v)`, signed features are represented relative to the orientation `u -> v`.


In [ ]:
num_signed_features = 3
num_unsigned_features = 6

# Represent the graph
edge_index = (
    torch.tensor(
        [
            [0, 1],
            [1, 2],
            [2, 3],
            [2, 4],
            [3, 5],
            [5, 0],
            [5, 2],
        ]
    )
    .t()
    .contiguous()
)
edge_is_directed = torch.tensor([0, 0, 0, 0, 0, 1, 1], dtype=torch.bool)

dummy_features_signed = torch.randn(edge_index.size(1), num_signed_features)
dummy_features_unsigned = torch.randn(edge_index.size(1), num_unsigned_features)


### The Magnetic Edge Laplacian

Our code materializes the magnetic operators at the core of EIGN using sparse matrices to scale to large graphs. By introducing a phase shift, these operators are complex-valued.

In [3]:
from eign import magnetic_edge_laplacian, magnetic_incidence_matrix

In [ ]:
L_signed_to_signed = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=True,
)
L_signed_to_unsigned = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=False,
)
L_unsigned_to_signed = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=False,
    signed_out=True,
)
L_unsigned_to_unsigned = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=False,
    signed_out=False,
)
L_signed_to_signed

/nfs/homedirs/fuchsgru/magnetic_edge_gnn/magnetic_edge_gnn_public/eign/laplacian.py:88: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  laplacian = (


tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4,
                        4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6],
                       [0, 1, 5, 0, 1, 2, 3, 6, 1, 2, 3, 4, 6, 1, 2, 3, 6, 2, 4,
                        5, 6, 0, 4, 5, 6, 1, 2, 3, 4, 5, 6]]),
       values=tensor([ 2.0000+0.0000j, -1.0000+0.0000j, -0.9010+0.4339j,
                      -1.0000+0.0000j,  2.0000+0.0000j, -1.0000+0.0000j,
                      -1.0000+0.0000j,  0.9010-0.4339j, -1.0000+0.0000j,
                       2.0000+0.0000j,  1.0000+0.0000j, -1.0000+0.0000j,
                      -0.9010+0.4339j, -1.0000+0.0000j,  1.0000+0.0000j,
                       2.0000+0.0000j, -0.9010+0.4339j, -1.0000+0.0000j,
                       2.0000+0.0000j, -0.9010-0.4339j, -0.9010-0.4339j,
                      -0.9010-0.4339j, -0.9010+0.4339j,  2.0000+0.0000j,
                       1.0000+0.0000j,  0.9010+0.4339j, -0.9010-0.4339j,
                      -0.9010-0.4339j, -0.9010+0.4339j,  

These operators can be used as (orientation equivariant / invariant) graph shift operator for edge signals

In [5]:
L_signed_to_signed @ dummy_features_signed.to(L_signed_to_signed.dtype)
L_signed_to_unsigned @ dummy_features_signed.to(L_signed_to_unsigned.dtype)
L_unsigned_to_unsigned @ dummy_features_unsigned.to(L_unsigned_to_unsigned.dtype)
L_unsigned_to_signed @ dummy_features_unsigned.to(L_unsigned_to_signed.dtype)

tensor([[ 0.8217-0.1546j, -1.1334+0.4762j, -1.9667+0.4319j,  0.4545-0.2535j,
         -0.3153-0.1828j, -0.3012-0.1783j],
        [ 2.4587-0.3658j, -0.7512+0.0724j, -4.0248+0.0471j, -0.4232+0.5641j,
         -0.1102-0.6852j,  1.8796+0.0078j],
        [ 0.3097+0.3658j, -1.1638-0.0724j,  1.2476-0.0471j,  0.2748-0.5641j,
          0.4837+0.6852j,  1.6423-0.0078j],
        [-1.8372+0.3658j, -0.2815-0.0724j,  3.7211-0.0471j,  2.3676-0.5641j,
         -0.8315+0.6852j, -0.5757-0.0078j],
        [-0.1383+0.2112j,  0.2619+0.4037j,  3.3524+0.3848j, -0.5735-0.8176j,
          0.9397+0.5024j, -1.6497-0.1861j],
        [-3.2686+0.3093j,  2.5420+0.1807j,  1.2741-0.3825j,  0.4857+0.0797j,
         -1.1964+0.6635j, -1.4121-0.2565j],
        [-0.0873+1.2638j,  0.6060-0.1427j, -4.3318-2.5505j,  0.3791+0.3732j,
         -1.2031-0.3032j,  0.0840+0.2748j]])

Be mindful that for `q != 0`, these shift operators are complex-valued and so must their inputs. A convenient way to combine this with learnable feature transformations that operate on real values is to flatten and unflatten the edge signal. Effectively, we therefore model the real and complex part of the signals.

In [ ]:
num_signed_features_out = 16
lin = nn.Linear(num_signed_features, num_signed_features_out)

hidden_signed_complex = torch.view_as_complex(
    lin(dummy_features_signed).reshape(-1, num_signed_features_out // 2, 2)
)
hidden_signed_complex = L_signed_to_signed @ hidden_signed_complex
hidden_signed = torch.view_as_real(hidden_signed_complex).reshape(
    -1, num_signed_features_out
)
hidden_signed.size()

torch.Size([7, 16])

### Node-level signals

The magnetic incidence matrices define mappings from edge signals to node signals and vice versa. This can be used to incorporate or predict node level outputs while still satisfying orientation equivariance or invariance.

These node level signals are invariant to orientation changes of *undirected* edges. Therefore, any transformation (that does not depend on orientation) can be used to transform them, if desired (see `eign_example.ipynb`).

In [7]:
B_signed = magnetic_incidence_matrix(
    edge_index, edge_is_directed, q=1 / edge_index.size(1), signed=True
)

In [8]:
node_level = B_signed @ hidden_signed.to(B_signed.dtype)
node_level.size()

torch.Size([6, 16])

Alternatively, the boundary operator can transform node level signals to the edge level.

In [ ]:
node_level = torch.randn(
    int(edge_index.max().item()) + 1, num_signed_features_out, dtype=B_signed.dtype
)
edge_level = B_signed.conj().t() @ node_level
edge_level.size()

torch.Size([7, 16])

### Built-in Convolution operators

The `eign` package also provides convolution operators that use these Laplacians. Each of these convolutions uses a single linear layer as an edge-feature transformation.

In [ ]:
from eign import (
    MagneticEdgeLaplacianConv,
    MagneticEdgeLaplacianWithNodeTransformationConv,
)

In [ ]:
conv = MagneticEdgeLaplacianConv(
    num_signed_features,
    num_signed_features_out,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=True,
    bias=None,  # Setting `bias` to `None` automatically uses a bias only if orientation equivariance / invariance is preserved
).eval()
conv

MagneticEdgeLaplacianConv(
  (lin): Linear(in_features=3, out_features=16, bias=False)
)

In [12]:
conv(dummy_features_signed, edge_index, edge_is_directed)

tensor([[-0.1947,  0.0541, -0.0250,  0.0694,  0.1348, -0.1446, -0.0823,  0.0922,
          0.2742, -0.0613,  0.1299,  0.2300, -0.2642,  0.1088, -0.3816, -0.1783],
        [ 0.1571,  0.4101,  0.1678, -0.1374, -0.3512, -0.0404,  0.0118, -0.2674,
         -0.3869, -0.4587, -0.3849, -0.1212,  0.0886,  0.3370,  0.0744,  0.2161],
        [-0.0665, -0.6434, -0.2062,  0.1675,  0.4068,  0.1216,  0.0375,  0.3975,
          0.3628,  0.7694,  0.6348,  0.0207,  0.0916, -0.5622,  0.0746, -0.1186],
        [-0.3214, -0.4906, -0.2058,  0.2048,  0.5075, -0.0204, -0.0403,  0.3333,
          0.6263,  0.5424,  0.4678,  0.2256, -0.2726, -0.4103, -0.2229, -0.4124],
        [-0.2361,  0.4843,  0.1392, -0.0660, -0.1332, -0.1894, -0.0702, -0.3535,
          0.0480, -0.6256, -0.6313,  0.1122, -0.4302,  0.3780, -0.2138, -0.2889],
        [ 0.2711, -0.2641, -0.0482, -0.0132, -0.0656,  0.1912,  0.1559,  0.1288,
         -0.2151,  0.2895,  0.2627, -0.3256,  0.4530, -0.2298,  0.4074,  0.4012],
        [ 0.0450,  0.1

You can also define your own node-level feature transformation. For example, you could inform the convolution about node features that are available.

In [ ]:
class MyNodeTransformation(nn.Module):
    def __init__(
        self, num_features_in_edge, num_features_out_edge, num_features_in_node
    ):
        super().__init__()
        self.edge_level = nn.Linear(num_features_in_edge, num_features_out_edge)
        self.node_level = nn.Linear(num_features_in_node, num_features_out_edge)

    def forward(self, edge_level, node_level):
        return F.relu(self.edge_level(edge_level) + self.node_level(node_level))


def initialize_node_feature_transformation(
    num_features_in_edge, num_features_out_edge, num_features_in_node
):
    return MyNodeTransformation(
        num_features_in_edge, num_features_out_edge, num_features_in_node
    )


conv = MagneticEdgeLaplacianWithNodeTransformationConv(
    num_signed_features,
    num_signed_features_out,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=True,
    initialize_node_feature_transformation=initialize_node_feature_transformation,
    # Additional args and kwargs are passed to `initialize_node_feature_transformation`
    num_features_in_node=13,
).eval()

node_level_signal = torch.randn(int(edge_index.max().item()) + 1, 13)
conv(
    dummy_features_signed,
    edge_index,
    edge_is_directed,
    # Additional args and kwargs are passed to `initialize_node_feature_transformation.forward`
    node_level_signal,
)

tensor([[ 0.1717, -0.1023,  0.5849,  0.0000, -0.3196,  0.0000, -0.6225, -0.0357,
          0.0000,  0.2613,  0.2235, -0.0785, -0.0319, -0.1055,  0.0000,  0.3445],
        [-0.2033,  0.0000, -0.5510,  0.0013,  0.2118,  0.0000, -0.1282,  0.0000,
          0.1729, -0.0098, -0.1977, -0.0457,  0.0000, -0.0332,  0.1142, -0.3022],
        [-0.0543,  0.2081, -0.0095, -0.0013, -0.0286,  0.0000,  0.2898,  0.0000,
         -0.1729, -0.2035,  0.0000,  0.0000,  0.3890,  0.0452, -0.1078,  0.1043],
        [ 0.1211,  0.1954,  0.2030, -0.0015, -0.2320,  0.0000,  0.0518,  0.2089,
         -0.1894, -0.2230,  0.7536,  0.1175,  0.0855,  0.4584, -0.1251,  0.4523],
        [ 0.3745, -0.2280, -0.0725,  0.0000, -0.2007,  0.0000, -0.2536,  0.0000,
          0.1577,  0.1490,  0.0000,  0.0000, -0.2078, -0.0757,  0.3208,  0.3185],
        [-0.2613,  0.3007,  0.1472,  0.0709,  0.2576,  0.1240,  0.5568,  0.3590,
         -0.2067, -0.0658, -0.0371,  0.1156, -0.2805,  0.2486, -0.4931, -0.2270],
        [-0.2399,  0.1

### Using EIGN Blocks

One block of EIGN uses convolutions and a fusion layer to model local interactions. This is also encapsulated in a block class that can be used if so desired.

In [ ]:
from eign import (
    EIGNBlockMagneticEdgeLaplacianConv,
    EIGNBlockMagneticEdgeLaplacianWithNodeTransformationConv,
)

In [15]:
num_unsigned_features_out = 16

In [16]:
block = EIGNBlockMagneticEdgeLaplacianConv(
    num_signed_features,
    num_signed_features_out,
    num_unsigned_features,
    num_unsigned_features_out,
    q=1 / edge_index.size(1),
).eval()
block

EIGNBlockMagneticEdgeLaplacianConv(
  (signed_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=False)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=False)
  )
  (unsigned_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=True)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=True)
  )
  (unsigned_conv): ResidualWrapper(
    (lin): Linear(in_features=6, out_features=16, bias=False)
    (conv): MagneticEdgeLaplacianConv(
      (lin): Linear(in_features=6, out_features=16, bias=False)
    )
  )
  (unsigned_signed_conv): MagneticEdgeLaplacianConv(
    (lin): Linear(in_features=6, out_features=16, bias=False)
  )
  (signed_conv): ResidualWrapper(
    (lin): Linear(in_features=3, out_features=16, bias=False)
    (conv): MagneticEdgeLaplacianConv(
      (lin): Linear(in_features=3, out_features=16, bias=False)
    )
  )
  (signed_unsigned_conv): MagneticEdgeLaplacianConv(
    (lin

In [19]:
output_signed, output_unsigned = block(
    dummy_features_signed, dummy_features_unsigned, edge_index, edge_is_directed
)

In [20]:
output_signed

tensor([[ 0.3385, -0.6182, -0.0098,  0.0614,  0.6184,  0.3665, -0.4716, -0.0068,
         -0.1353, -0.0816, -0.4102,  0.4001, -0.4370,  0.3387,  0.2809,  0.5173],
        [ 0.0744, -0.6355,  0.2281,  0.3918,  0.3793, -0.1209, -0.1914, -0.3636,
         -0.3928,  0.0359, -0.4980,  0.6236,  0.4621,  0.1033, -0.7377,  0.3394],
        [ 0.2017, -0.2011, -0.5986,  0.2791,  0.2830,  0.2737, -0.5414, -0.2167,
          0.0325, -0.5175,  0.4129,  1.0696, -0.1149,  0.5954,  0.0519,  0.4444],
        [ 0.1161,  0.1385, -0.8816, -0.3141, -0.5899,  0.6027,  0.4899, -0.3703,
          0.7215, -0.3942,  0.2784, -0.8480, -0.6390, -0.3134,  0.6778,  0.3340],
        [-0.4231,  0.7605, -0.1806,  0.4953,  0.2423, -0.3945,  0.3326,  0.7699,
          0.1064,  0.0889,  0.5313, -0.5098, -0.1316,  0.2270,  0.6311, -0.6635],
        [-0.5779, -0.1517,  1.0190, -0.4238,  0.4227,  0.2474, -1.0532,  0.8054,
         -0.2241, -0.4060,  0.8906,  0.8297,  0.3009,  0.1970,  0.6101, -0.4322],
        [-0.1922, -0.1

In [21]:
output_unsigned

tensor([[ 5.2657e-02, -7.0489e-02,  2.0003e-01, -2.0070e-03,  7.2753e-01,
          3.2055e-02, -2.3380e-02,  2.8872e-02,  1.1404e-01,  1.0499e+00,
          5.2268e-02,  1.1564e+00, -9.3413e-03,  7.8790e-03,  3.0992e-01,
          3.7241e-02],
        [ 9.7066e-01,  3.9173e-01,  5.8379e-01,  3.5099e-01,  3.0039e-01,
          2.8125e-01,  1.0051e+00,  1.0953e-01,  3.8053e-01, -1.0440e-01,
          3.3063e-01,  2.4506e-01,  1.6649e-03,  1.0298e-01, -9.7932e-02,
          8.4382e-01],
        [ 1.5437e+00,  7.7958e-03,  9.2560e-01,  8.8244e-01,  1.4747e+00,
          8.3546e-01,  1.8164e+00,  2.7799e-01,  3.0769e-02, -1.5405e-01,
          7.1727e-01,  1.0116e+00, -8.3047e-03,  1.0763e-01, -5.4786e-02,
          7.1665e-01],
        [ 1.1498e+00,  9.7659e-01,  8.2291e-01,  5.6469e-01,  7.8259e-02,
          7.9976e-01,  7.2744e-01,  2.8601e-02,  8.9988e-01,  2.6680e-01,
          1.2892e+00,  1.8084e-01,  2.5511e-01,  4.4877e-02, -3.4934e-01,
          4.0353e-01],
        [ 1.3831e+00

Again, you can also learn node feature representations induced by the edge signals.

In [ ]:
block = EIGNBlockMagneticEdgeLaplacianWithNodeTransformationConv(
    num_signed_features,
    num_signed_features_out,
    num_unsigned_features,
    num_unsigned_features_out,
    q=1 / edge_index.size(1),
    initialize_node_feature_transformation=lambda num_in, num_out: nn.Sequential(
        nn.ReLU(), nn.Linear(num_in, num_out)
    ),
).eval()
block

EIGNBlockMagneticEdgeLaplacianWithNodeTransformationConv(
  (signed_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=False)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=False)
  )
  (unsigned_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=True)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=True)
  )
  (unsigned_conv): ResidualWrapper(
    (lin): Linear(in_features=6, out_features=16, bias=False)
    (conv): MagneticEdgeLaplacianWithNodeTransformationConv(
      (lin): Linear(in_features=6, out_features=16, bias=False)
      (node_feature_transformation): Sequential(
        (0): ReLU()
        (1): Linear(in_features=16, out_features=16, bias=True)
      )
    )
  )
  (unsigned_signed_conv): MagneticEdgeLaplacianWithNodeTransformationConv(
    (lin): Linear(in_features=6, out_features=16, bias=False)
    (node_feature_transformation): Sequential(
      (0): ReLU()
  

In [25]:
output_signed, output_unsigned = block(
    dummy_features_signed, dummy_features_unsigned, edge_index, edge_is_directed
)